## Gradient Checking

In [1]:
def dictionary_to_vector(params_dict):
    L = len(layers_dims)
    parameters = {}
    k = 0
    for l in range(1, L):
    # Create temp variable to store dimension used on each layer
    w_dim = layers_dims[l] * layers_dims[l - 1]
    b_dim = layers_dims[l]
    # Create temp var to be used in slicing parameters vector
    temp_dim = k + w_dim
    # add parameters to the dictionary
    parameters["W" + str(l)] = vector[ k:temp_dim].reshape(layers_dims[l], layers_dims[l - 1]) 
    parameters["b" + str(l)] = vector[ temp_dim:temp_dim + b_dim].reshape(b_dim, 1)
    k += w_dim + b_dim
    return parameters


def gradients_to_vector(gradients):
# Get the number of indices for the gradients to iterate over valid_grads = [key for key in gradients.keys() if not key.startswith("dA")]
    L = len(valid_grads)// 2
    count = 0
    # Iterate over all gradients and append them to new_grads list
    for l in range(1, L + 1):
    if count == 0:
    new_grads = gradients["dW" + str(l)].reshape(-1, 1)
    new_grads = np.concatenate((new_grads, gradients["db" + str(l)].reshape(-1, 1)))
    else:
    new_grads = np.concatenate((new_grads, gradients["dW" + str(l)].reshape(-1, 1)))
    new_grads = np.concatenate( (new_grads, gradients["db" + str(l)].reshape(-1, 1)))
    count += 1
    return new_grads

def forward_prop_cost(X, parameters, Y, hidden_layers_activation_fn="tanh"):
# Compute forward prop
    AL, _ = L_model_forward(X, parameters, hidden_layers_activation_fn) # Compute cost
    cost = compute_cost(AL, Y)
    return cost 


def gradient_check( parameters, gradients, X, Y, layers_dims, epsilon=1e-7, hidden_layers_activation_fn="tanh"):
    # Roll out parameters and gradients dictionaries
    parameters_vector = dictionary_to_vector(parameters) gradients_vector = gradients_to_vector(gradients)
    # Create vector of zeros to be used with epsilon
    grads_approx = np.zeros_like(parameters_vector)
    for i in range(len(parameters_vector)):
    # Compute cost of theta + epsilon
    theta_plus = np.copy(parameters_vector)
    theta_plus[i] = theta_plus[i] + epsilon
    j_plus = forward_prop_cost( X, vector_to_dictionary(theta_plus, layers_dims), Y, hidden_layers_activation_fn)
    # Compute cost of theta - epsilon
    theta_minus = np.copy(parameters_vector)
    theta_minus[i] = theta_minus[i] - epsilon
    j_minus = forward_prop_cost( X, vector_to_dictionary(theta_minus, layers_dims), Y, hidden_layers_activation_fn)
    # Compute numerical gradients
    grads_approx[i] = (j_plus - j_minus) / (2 * epsilon)
    # Compute the difference of numerical and analytical gradients numerator = norm(gradients_vector - grads_approx)
    denominator = norm(grads_approx) + norm(gradients_vector)
    difference = numerator / denominator
    if difference > 10e-7:
    print ("\033[31mThere is a mistake in back-propagation " +\ "implementation. The difference is: {}".format(difference))
    else:
    print ("\033[32mThere implementation of back-propagation is fine! "+\ "The difference is: {}".format(difference))
    return difference

IndentationError: expected an indented block (<ipython-input-1-bba56ab16aca>, line 7)

## RSMP

$$ S_{\frac{\partial{loss}}{{\partial{w}}}} = \beta S_{\frac{\partial{loss}}{{\partial{w}}}} + (1 - \beta) ||{\frac{\partial{loss}}{{\partial{w}}}}||^2 $$
$$ S_{\frac{\partial{loss}}{{\partial{b}}}} = \beta S_{\frac{\partial{loss}}{{\partial{b}}}} + (1 - \beta) ||{\frac{\partial{loss}}{{\partial{b}}}}||^2 $$
$$ w = w - \alpha \frac {\frac{\partial{loss}}{\partial{w}}} {\sqrt{S_{\frac{\partial{loss}}{{\partial{w}}}}}} $$
$$ b = b - \alpha \frac {\frac{\partial{loss}}{\partial{b}}} {\sqrt{S_{\frac{\partial{loss}}{{\partial{b}}}}}} $$

## Adam

$$ V_{dw} = \beta_1 V_{dw} + (1 - \beta _ 1) dW $$
$$ V_{db} = \beta_1 V_{db} + (1 - \beta _ 1) db $$
$$ S_{dw} = \beta_2 S_{dw} + (1 - \beta _ 2) ||dw||^2 $$
$$ S_{db} = \beta_2 S_{db} + (1 - \beta _ 2) ||db||^2 $$

$$ V_{dw}^{corrected} = \frac{V_{dw}} {1 - \beta_1^{t}} $$ 
$$ V_{db}^{corrected} = \frac{V_{db}} {1 - \beta_1^{t}} $$ 

$$ S_{dw}^{corrected} = \frac{S_{dw}} {1 - \beta_2^{t}} $$ 
$$ S_{db}^{corrected} = \frac{S_{db}} {1 - \beta_2^{t}} $$ 

$$ W = W - \alpha \frac{V_{dw}^{corrected} } {\sqrt{S_{dw}^{corrected}} + \epsilon} $$
$$ b = b - \alpha \frac{V_{db}^{corrected} } {\sqrt{S_{db}^{corrected}} + \epsilon} $$